In [4]:
import pandas as np
import numpy as np
import os
import tensorflow as tf

In [167]:
%load_ext autoreload
%autoreload 2

from loader import *
from converter import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get Data start end for each person

In [6]:
sbj = pd.read_csv('subjects.csv')

In [8]:
pids = list(range(1,33))
startEndDic={}

for i in range(32):
    startEndDic[i+1] = [ sbj.iloc[i]['startTime'], sbj.iloc[i]['endTime'] ] 

In [66]:
start, end = startEndDic[1]

## Generate .npy file for each person 

In [82]:
cols = ['timestamp', 'delta', 'lowAlpha', 'highAlpha', 'lowBeta', 'highBeta', 'lowGamma', 'middleGamma', 'theta']

for pid in pids:
    print(pid)
    # get start, end
    start, end = startEndDic[pid]
    
    #load data & label 
    data, label = pd.read_csv(str(pid)+'_neuro.csv'), pd.read_csv(str(pid)+'_label.csv')
    
    #transfor
    label = label_converter(label, 'arousal', [1,1,1], [2.9,3.1])
    data = match_ts(start, end, data, cols)
    
    #save npy
    np.save(str(pid)+'_x.npy',data)
    np.save(str(pid)+'_y.npy',label)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [88]:
datas = []
for i in pids:
    datas.append(np.load(str(i)+'_x.npy'))

In [89]:
labels = []
for i in pids:
    labels.append(np.load(str(i)+'_y.npy'))

In [90]:
len(datas), len(labels)

(32, 32)

In [104]:
datas[0][0]

array([[ 151358.,   77237.,   53420.,   77722.,    9556.,    7736.,
        1453364.,  813273.],
       [  18626.,   36372.,   21400.,   55311.,    2814.,    4741.,
        1079635.,  134751.],
       [   8023.,   36135.,   11053.,   10367.,    2366.,    1512.,
         101985.,   60068.],
       [  36935.,   46703.,   93904.,  105495.,    7960.,    7016.,
         992061.,  421516.],
       [  28364.,  148108.,   36245.,   67606.,    4933.,    7927.,
         594838.,  595962.]])

In [105]:
#Normalize each data/person
for i in range(len(datas)):
    d= datas[i]
    datas[i] = (d-d.min())/(d.max()-d.min())

In [106]:
datas[0][0]

array([[4.38055414e-02, 2.23297030e-02, 1.54289590e-02, 2.24702270e-02,
        2.71979190e-03, 2.19246461e-03, 4.21049109e-01, 2.35588971e-01],
       [5.34773613e-03, 1.04894669e-02, 6.15147563e-03, 1.59768579e-02,
        7.66363009e-04, 1.32469251e-03, 3.12764768e-01, 3.89938248e-02],
       [2.27562007e-03, 1.04207985e-02, 3.15353308e-03, 2.95477125e-03,
        6.36559369e-04, 3.89121180e-04, 2.95001951e-02, 1.73551523e-02],
       [1.06525907e-02, 1.34827736e-02, 2.71588040e-02, 3.05171835e-02,
        2.25736643e-03, 1.98385162e-03, 2.87391054e-01, 1.22081193e-01],
       [8.16922686e-03, 4.28638855e-02, 1.04526699e-02, 1.95392145e-02,
        1.38032264e-03, 2.24780500e-03, 1.72299556e-01, 1.72625224e-01]])

In [107]:
#split by user
trainData, testData = datas[:24], datas[24:]
trainY, testY = labels[:24], labels[24:]

In [110]:
# 6*5 sec for each train
win=6

#majority for each label
trainY[:2]

[array([0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1], dtype=int8),
 array([1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 

In [114]:
yArr = trainY[0]

In [152]:
l = len(yArr)
th = 2
arr = []
win = 6
for i in range(l-win+1):
    y = yArr[i:i+win]
    if np.count_nonzero(y == 2) >= th:
        arr.append(2)
    elif np.count_nonzero(y == 0) >= th:
        arr.append(0)
    else:
        arr.append(1)
        

In [161]:
l = len(xArr)
th = 1
arr = []
win = 6
for i in range(l-win+1):
    x = xArr[i:i+win]
    x = np.concatenate(x)
    arr.append(x)        

In [164]:
np.array(arr)[0].shape

(30, 8)

In [169]:
len(data_window(xArr, win))

165